In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, TargetEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split

In [ ]:
# Load the OKCupid data
# Downloaded on January 18 from https://github.com/rudeboybert/JSE_OkCupid/blob/master/profiles_revised.csv.zip

df = pd.read_csv("profiles_revised.csv")

In [ ]:
df.info()

In [ ]:
# before we go too far, split!
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# How many values in each category?
print("Feature     Count")
for feat in train.select_dtypes(include="object").columns:
    print(f"{feat:12}{train[feat].nunique()}")

In [ ]:
# look at drinks category

In [ ]:
# Drop nans for demonstration purposes
nona_speaks = train["speaks"].dropna()
encoder = FeatureHasher(n_features=8, input_type="string")
encoded_speaks = encoder.fit_transform(nona_speaks.str.split(", "))

In [ ]:
for s, e in zip(nona_speaks[:5], encoded_speaks[:5]):
    print(f"{s}: {e.todense()}")